#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "XXXX"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備

言語理解でのプロンプト例

In [ ]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
2. 出力は指示に対する適切な応答であるべきです。出力は3か2か1か0であることを確認してください。
3. 生成するinputとoutputは全て必ず日本語で生成してください。
4. 文法的に正しいデータと誤りを含んでいるデータを5:5でバランス良く生成してください。
5. 誤りを含むデータについては、多様な誤りを含むようにしてください。難し過ぎる誤りではなく、ぱっと見で分かるレベルの誤りを含んでください。
6. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "<入力文章>",
    "output": "<3 or 2 or 1 or 0>"
  },
  {
    "id": 2,
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "<入力文章>",
    "output": "<3 or 2 or 1 or 0>"
  },
  {
    "id": 3,
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "<入力文章>",
    "output": "<3 or 2 or 1 or 0>"
  }
]
"""

In [ ]:
FEWSHOT_EXAMPLES = [
	{
		"id" : 1,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "個別の株価が市場全体の値動きに対して、どの程度連動しているかを示すリスク指標として正しいのはどれですか？0: α, 1: β, 2: γ, 3: θ",
		"output" : "1"
	},
	{
		"id" : 2,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "一つの選挙区から一人を選出する選挙制度はなんと呼ばれますか？, 0: 大選挙区制, 1: 中選挙区制, 2: 小選挙区制, 3: 議会制民主主義",
		"output" : "2"
	},
	{
		"id" : 3,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "企業において、主にマーケティングについて統括する最高責任者はなんと呼ばれますか？0: CEO, 1: CTO, 2: CMO, 3: CFO",
		"output" : "2"
	},
	{
		"id" : 4,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "マーケティング・ミックスのフレームワークである4Pに含まれない要素はどれですか？0: Product, 1: Price, 2: Place, 3: Persistent",
		"output" : "3"
	},
	{
		"id" : 5,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "顧客との契約の締結にあたる営業活動の終盤のステップをなんと呼びますか？0: クロージング, 1: ディスクロージャー, 2: ヒアリング, 3: フィールドセールス",
		"output" : "0"
	},
	{
		"id" : 6,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "機能やサービスを提供するPCとそれを利用するPCを分離して、それらをネットワーク通信で接続するソフトウェアアーキテクチャをなんと呼びますか？0: ピアツーピアモデル, 1: クライアントサーバモデル, 2: メインフレームモデル, 3: カスケードモデル",
		"output" : "1"
	},
	{
		"id" : 7,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "企業の自己資本に対する当期純利益の割合をなんといいますか？0: ROE, 1: ROA, 2: BPS, 3: PER",
		"output" : "0"
	},
	{
		"id" : 8,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "自社のサービスや製品について、他社のものと区別するために使用されるマークやネーミングをなんという？0: 特許, 1: プライバシーマーク, 2: ISO, 3: 商標",
		"output" : "3"
	},
	{
		"id" : 9,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "会計上は無形固定資産として扱われ、企業の買収額と時価評価純資産の差額をなんという？0: 特許権, 1: 意匠権, 2: 実用新案権, 3: のれん",
		"output" : "3"
	},
	{
		"id" : 10,
		"instruction" : "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
		"input" : "上司だけでなく、部下や同僚など複数人で対象者を評価する人事制度をなんという？0: コンピテンシー評価, 1: 360度評価, 2: 目標管理, 3: コーチング",
		"output" : "1"
	}
]

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE=1 #temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS=10000
LOOP_NUM = 100 #1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。"},
         {"role": "user", "content": PROMPT_TEMPLATE}],
        temperature=TEMPERATURE,#temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,#0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,#-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3 #-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
        )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            #output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

ストリーミング出力は最後の 5000 行に切り捨てられました。
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "社会的なことを考える時、多くの人の意見を集める手法は何ですか？0: アンケート, 1: アンケート, 2: 調査, 3: サーベイ",
    "output": "0"
  },
  {
    "id": 3,
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "正確性を保証することが求められる分野では、証拠を重視した論法を用いるべきです。それを表現する法則は何ですか？0: 高低法則, 1: 成立法則, 2: 証拠法則, 3: 前後法則",
    "output": "2"
  },
  {
    "id": 4,
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "心理学において、感情を理解するためには、身体や顔の表情、言葉などの相手の行動を知ることが重要です。それぞれの分野は何ですか？0: 感情認知, 1: 身体認知, 2: 顔認知, 3: 言葉認知",
    "output": "0"
  },
  {
    "id": 5,
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "二つの車が等速直線運動をしていますが、一方の車がもう一方の車に追いつく場合、その距離と速度はどのような関係にありますか？0: 等速直線運動の関係, 1: 比例関係, 2: 反比例関係, 3: 比例増加関係",
    "output": "1"
  },
  {
    "id": 6,
    "instruction": "以下の質問に対し回答してください。回答は選択肢の番号でしてください。",
    "input": "両手で物を持つ時、その物が重いときにはどのような姿勢を取るべきですか？0: 腰を低くする, 1: 腰を高くする, 2: 背中を曲げる, 3: 肩を高くする",
    "output

,id,instruction,input,output
0,1,以下の質問に対し回答してください。回答は選択肢の番号でしてください。,"法人税の計算において使用される、税務上の所得額を表す金額は何ですか？0: 課税所得, 1: ...",0
1,2,以下の質問に対し回答してください。回答は選択肢の番号でしてください。,"電子メールの著作権を表すマークは何ですか？0: ©, 1: ®, 2: ™, 3: Ⓟ",0
2,3,以下の質問に対し回答してください。回答は選択肢の番号でしてください。,"金融商品取引法に基づく投資信託の取引所は何ですか？0: TSE, 1: SSE, 2: TF...",2
3,4,以下の質問に対し回答してください。回答は選択肢の番号でしてください。,市場調査の手法で、少数のrespondentから情報を収集するものは何ですか？0: 質的調査...,3
4,5,以下の質問に対し回答してください。回答は選択肢の番号でしてください。,"ファーストムーバーの特徴として正しいものは何ですか？0: 顧客の品質保証, 1: 高い利益率...",2


##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")

File saved as: /generated_data_2024-05-23_11-14-44.xlsx
